# Coston 2 custom-node (RPC only)

In [1]:
from pathlib import Path
import pandas as pd
import json

data_dir = Path("..") / "data" / "derived"

In [2]:
# Read the Parquet files 
df_blocks   = pd.read_parquet(data_dir / "df_blocks.parquet")
df_snowball = pd.read_parquet(data_dir / "df_snowball.parquet")

In [11]:
print(f"Parsed {len(df_blocks)} blocks from Coston 2.")

Parsed 212228 blocks from Coston 2.


In [23]:
from collections import Counter

slot_gt0 = df_blocks.query("slot > 0")

# remove self before counting
clean_prev = slot_gt0.apply(
    lambda r: [p for p in r.prevProposers if p != r.proposerID],
    axis=1
)

proposer_counts = clean_prev.explode().value_counts()

print("Rows where proposerID appeared in prevProposers:",
      (slot_gt0.apply(lambda r: r.proposerID in r.prevProposers, axis=1)).sum())

proposer_counter 

Rows where proposerID appeared in prevProposers: 3800


Counter({'NodeID-FQKTLuZHEsjCxPeFTFgsojsucmdyNDsz1': 3882,
         'NodeID-87skCc745aTkJEtozqTJ2SBnZ8Vb8Ncbd': 3306,
         'NodeID-K6pAWivFKy38hCZSd4epCt7TW4fzXz7kP': 3253,
         'NodeID-2mCUTA33bHU4vb8NQHE8QewftqC8p1fv8': 3208,
         'NodeID-CZYx3on11wwYXFoHwZtAQZT5unZ9JHMf6': 3199,
         'NodeID-AKt7WaK6ozEy5K8azKNacZXLzxZ9xFgC7': 3114,
         'NodeID-ECqqsWu9jpUvZgPyf9yPZZcyWmJjWyN1T': 968})

In [29]:
# Snowball count per block heigh
snowball_per_height = (
    df_snowball.groupby("height").size()
)

df_blocks["snowball_blocks"] = (
    df_blocks["height"]
      .map(snowball_per_height)
      .fillna(0)
      .astype(int)
)

In [34]:
print(f"Parsed a total of {len(df_blocks)} blocks.")

slot0 = len(df_blocks[df_blocks['slot'] != 0])
print(f"Instances where block was NOT created in first slot: {slot0}")
      
multiple_snowballs = len(df_blocks[df_blocks["snowball_blocks"] > 1])
print(f"Instances where multiple blocks were proposed at same height: {multiple_snowballs}.")

Parsed a total of 212228 blocks.
Instances where block was NOT created in first slot: 17706
Instances where multiple blocks were proposed at same height: 28.


In [49]:
df_blocks[(df_blocks["snowball_blocks"] > 1) & (df_blocks['slot'] == 0)]
# this could only happen if the proposer actually proposed multiple in the initial slot
# TODO: add gas usage, # of txs for each proposed block
# per-block polls and consensus-metrics

,blkID,height,pChainHeight,proposerID,slot,firstSlot,prevProposers,snowball_blocks
7758,8YmHVLyfzfPY2UgaL5RZQjvv6U4JaDwrRXLLpxUbExozXmVqw,19881243,9911,NodeID-87skCc745aTkJEtozqTJ2SBnZ8Vb8Ncbd,0,True,[],2
17000,SwLXLP8WR7nLEX1m6oaubbbbQaybe1r5wg3GHVjq8TMGsBPof,19890616,9916,NodeID-FQKTLuZHEsjCxPeFTFgsojsucmdyNDsz1,0,True,[],2
20564,EuDNVpc5gVBY2pRS1VfrHQzHLjao2cgFgLt3Egz53Tj5QZDXg,19894973,9916,NodeID-CZYx3on11wwYXFoHwZtAQZT5unZ9JHMf6,0,True,[],2
26979,2KNP2X4Evdeemgd4L11VtkQgmPkm2dXhsskxWG8qMW1Te7...,19900056,9916,NodeID-FQKTLuZHEsjCxPeFTFgsojsucmdyNDsz1,0,True,[],2
70199,E5j6j2i51F4ZanJVuXyhbdwsoeLfpqYae9WfXfTunaia1CDR3,19943408,9925,NodeID-FQKTLuZHEsjCxPeFTFgsojsucmdyNDsz1,0,True,[],2
91145,2kBomcs9LGg7uEm3AScoTr9vVazDV2KwN2gBe8ZSqT3fBU...,19964689,9925,NodeID-87skCc745aTkJEtozqTJ2SBnZ8Vb8Ncbd,0,True,[],2
96541,BdF4VcvUqAGiLEBDWPDkRxXawwUHBFfzmUyAjVZe6kAgY8DPi,19969847,9925,NodeID-FQKTLuZHEsjCxPeFTFgsojsucmdyNDsz1,0,True,[],2
104207,tE6rBe1TzyriB7x6CnjDZXuu9Nkk1fjrruZQPi8kHWkcfRHd1,19977929,9925,NodeID-CZYx3on11wwYXFoHwZtAQZT5unZ9JHMf6,0,True,[],2
105282,JmwYF6pkGorVK6yktgyqRB4JZ18bTZNPF28GsuDrN22qrvoGy,19979510,9925,NodeID-87skCc745aTkJEtozqTJ2SBnZ8Vb8Ncbd,0,True,[],2
113192,2mtt6jMHR4sLuU5hTou3PdoAzdDMjJVcgeJzF6yK3jtiYG...,19987353,9925,NodeID-AKt7WaK6ozEy5K8azKNacZXLzxZ9xFgC7,0,True,[],2


In [52]:
df_blocks[df_blocks['height'] == 19881243+1]

,blkID,height,pChainHeight,proposerID,slot,firstSlot,prevProposers,snowball_blocks
7757,2hYSoVy8wzmTivzxf3Gu6SH8eC2a95Bkf73Ew5kbU1tRNV...,19881244,9911,NodeID-CZYx3on11wwYXFoHwZtAQZT5unZ9JHMf6,0,True,[],1


In [37]:
df_snowball[df_snowball['height'] == 19881243]

,blkID,height
4722,8YmHVLyfzfPY2UgaL5RZQjvv6U4JaDwrRXLLpxUbExozXmVqw,19881243
4723,2q7F9QvGZqRtZmoKpPdwWwkLaSVpiXcgn6THJcdBW8F5rT...,19881243


In [40]:
(4.271892e+06 - (20*213553)) / 28

29.714285714285715

In [2]:
#!/usr/bin/env python3
"""
Convert between Avalanche NodeID and Ethereum-style addresses.

Requirements (install in your own environment):
    pip install base58 ecdsa eth_utils

Usage:
    python convert_nodeid.py --nodeid NodeID-...      # get Ethereum address
    python convert_nodeid.py --privkey <hex>          # get NodeID from private key
"""

import argparse
import base58
from eth_utils import keccak, to_checksum_address
from ecdsa import VerifyingKey, SECP256k1, SigningKey

def nodeid_to_eth_address(node_id: str) -> str:
    """
    Convert an Avalanche NodeID (Base58Check) to an Ethereum-style address.
    """
    # Strip "NodeID-" prefix and Base58 decode with checksum
    b58 = node_id.split('-', 1)[1]
    decoded = base58.b58decode_check(b58)
    
    # Remove the network ID byte to get the compressed pubkey
    compressed_pk = decoded[1:]
    
    # Decompress the public key
    vk = VerifyingKey.from_string(compressed_pk, curve=SECP256k1)
    uncompressed_pk = b'\x04' + vk.to_string()
    
    # Ethereum address is last 20 bytes of keccak256(pubkey[1:])
    eth_address = to_checksum_address(keccak(uncompressed_pk[1:])[-20:])
    return eth_address

def privkey_to_nodeid(privkey_hex: str, network_id: int = 1) -> str:
    """
    Given a hex private key, derive and return the Avalanche NodeID.
    """
    sk = SigningKey.from_string(bytes.fromhex(privkey_hex), curve=SECP256k1)
    vk = sk.get_verifying_key()
    # Compress pubkey
    x = vk.pubkey.point.x()
    y = vk.pubkey.point.y()
    prefix = b'\x02' if y % 2 == 0 else b'\x03'
    compressed_pk = prefix + x.to_bytes(32, 'big')
    
    # Prefix network ID and Base58Check encode
    to_encode = bytes([network_id]) + compressed_pk
    nodeid_b58 = base58.b58encode_check(to_encode).decode()
    return f"NodeID-{nodeid_b58}"

In [5]:
eth = nodeid_to_eth_address("NodeID-BkJnMrPihNduN2iDBAtF9Esr5ednzF7FF")
print("Ethereum address:", eth)


ValueError: Invalid checksum

In [ ]:
nid = privkey_to_nodeid(args.privkey)
print("Avalanche NodeID:", nid)